# 开了解LCEL
LCEL是LangChain帮助我们快速构建复杂的链式组件的基本语法，并且流处理，并且调用和日志处理都是可以开箱即用的。

## 基本案例开始（prompt+context+output parser）

(动态提示词模板+输出) 这个是最常见也是最基本调的用方式，下面我将用获得城市旅游信息作为案例

In [22]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.callbacks import get_openai_callback

# 提示词模板
prompt = ChatPromptTemplate.from_template("我想去{topic}旅行，我想知道这个地方有什么好玩的")

# gpt模型
model = ChatOpenAI(
    openai_api_key="sk-gRbZ9FJz2E7c7mwO5JOvp2u2rtoWoAbg12CxDy3Y25eLeDvd",
    openai_api_base="https://api.chatanywhere.tech",
    temperature=.7
                 )

# 输出模板
print("输出模板:",prompt)

# 输出解析器
output_parser = StrOutputParser()

chain = prompt | model | output_parser

result = chain.invoke({"topic": "广州"})

print("结果:",result)


输出模板: input_variables=['topic'] messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['topic'], template='我想去{topic}旅行，我想知道这个地方有什么好玩的'))]
结果: 广州是一座充满活力和魅力的城市，有很多值得一游的地方。以下是一些广州的旅游景点：

1. 广州塔：也被称为小蛮腰，是广州的标志性建筑之一，可以乘坐观光电梯俯瞰整个城市。

2. 珠江夜游：可以在珠江上乘船夜游，欣赏广州夜景，尤其是珠江新城的灯光秀。

3. 越秀公园：是广州最大的城市中心公园，有美丽的湖泊、花园和建筑，是散步和休闲的好去处。

4. 陈家祠：是一座保存完好的古代宅第建筑群，展示了精美的木雕、砖雕和石雕艺术。

5. 荔枝湾：是广州的老城区，有着悠久的历史和文化，可以感受当地的传统风情和特色小吃。

6. 白云山：是广州著名的山岳风景区，有着壮丽的自然风光和丰富的登山、徒步等户外活动。

除此之外，广州还有很多美食、购物和文化体验等等，可以让你的旅行更加丰富多彩。希望你在广州旅行愉快！


注意下面这个代码，它就是LCEL语法将不同的组件拼凑成一个链

In [23]:
chain = prompt | model | output_parser

**|** 符号类似于Unix管道操作符，它将不同组件链接在一起，将一个组件的输出作为下一个组件的输入。
在这个链条中，用户输入被传递到提示模板，然后提示模板的输出被传递到模型，接着模型的输出被传递到输出解析器。让我们分别看看每个组件，真正理解正在发生的事情。

### 1.Prompt
**prompt**是一个基础prompt模板，它能接受一个变量  **{topic}**，并生成一个最终**promptValue**。**promptValue**是一个包装完成的提示的对象，可以传递给LLM（接受字符串作为输入）或ChatModel（接受消息序列作为输入）。它可以与任何一种语言模型类型一起使用，因为它定义了生成**BaseMessages**和生成字符串的逻辑。

In [24]:
# promt模板与参数结合
promptValue = prompt.invoke({"topic":"云南"})
promptValue

ChatPromptValue(messages=[HumanMessage(content='我想去云南旅行，我想知道这个地方有什么好玩的')])

In [25]:
promptValue.to_messages()

[HumanMessage(content='我想去云南旅行，我想知道这个地方有什么好玩的')]

In [26]:
promptValue.to_string()

'Human: 我想去云南旅行，我想知道这个地方有什么好玩的'

### 2.Model
promptValue 然后传递给模型。在这种情况下，我们的模型是一个 ChatModel，意味着它将输出一个 BaseMessage。

In [27]:
message = model.invoke(promptValue)
message

AIMessage(content='云南是一个美丽而多样化的地方，有很多好玩的活动和景点。以下是一些云南旅行的推荐：\n\n1. 丽江古城：这是一个保存完好的古老城市，被列入世界文化遗产名录。游客可以漫步在古城的石板街道上，欣赏古老的建筑和传统的手工艺品。\n\n2. 玉龙雪山：这座雪山是丽江的地标之一，是一个徒步旅行者和登山者的天堂。你可以乘坐缆车到达山顶，欣赏壮丽的雪山风景。\n\n3. 三江并流：这是一个壮丽的自然景观，是金沙江、怒江和澜沧江三条大江的交汇处。游客可以乘坐竹筏漂流，欣赏壮观的峡谷景色。\n\n4. 腾冲温泉：这里有丰富的地热资源，是一个理想的温泉度假胜地。你可以在温泉中放松身心，享受大自然的恩赐。\n\n5. 云南民族村寨：云南有许多不同的少数民族，每个民族都有独特的传统文化和风俗。你可以到各个民族村寨去了解他们的生活方式，欣赏他们的民俗表演和手工艺品。\n\n这些只是云南旅行中的一小部分，你还可以去看看世界自然遗产元阳梯田，参加茶马古道徒步旅行等等。无论你对自然景观还是文化遗产感兴趣，云南都能满足你的需求。')

如果我们的模型是LLM，它会输出一个字符串。

In [30]:
from langchain.llms import OpenAI

llm = OpenAI(
    openai_api_key="sk-gRbZ9FJz2E7c7mwO5JOvp2u2rtoWoAbg12CxDy3Y25eLeDvd",
    openai_api_base="https://api.chatanywhere.tech",
    model="gpt-3.5-turbo-instruct"
    )
llm.invoke(promptValue)

NotFoundError: <html>
<head><title>404 Not Found</title></head>
<body>
<center><h1>404 Not Found</h1></center>
<hr><center>nginx</center>
</body>
</html>